# Analyzing Used Car Data from eBay Kleinanzeigen
**by Gerard Tieng**

In this project, we will use [this dataset](https://www.kaggle.com/orgesleka/used-cars-database/data) originally scraped from German auction site eBay Kleinanzeigen and uploaded to Kaggle to find common trends in used car sales including:

- Common listed brands
- Common listed mileage
- Common listed registration year

By the end of this project the following skills in Data Cleaning, and the Pandas and NumPy libraries will be demonstrated:

- Using Pandas to read and inspect CSV files.
- Dataframe usage and manipulation including subsets and column renaming
- Pandas summary functions to remove outliers and false data.

## Data Loading & Inspection

Let's begin with importing with our required libraries for this project, then reading in the CSV file:

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv("autos.csv", encoding="Latin-1")

Now, we'll take a peek as well as print a summary of the dataset using **Dataframe.head()** and **Dataframe.info()**:

In [2]:
df.head()

,dateCrawled,name,seller,offerType,price,abtest,vehicleType,yearOfRegistration,gearbox,powerPS,model,odometer,monthOfRegistration,fuelType,brand,notRepairedDamage,dateCreated,nrOfPictures,postalCode,lastSeen
0,2016-03-26 17:47:46,Peugeot_807_160_NAVTECH_ON_BOARD,privat,Angebot,"$5,000",control,bus,2004,manuell,158,andere,"150,000km",3,lpg,peugeot,nein,2016-03-26 00:00:00,0,79588,2016-04-06 06:45:54
1,2016-04-04 13:38:56,BMW_740i_4_4_Liter_HAMANN_UMBAU_Mega_Optik,privat,Angebot,"$8,500",control,limousine,1997,automatik,286,7er,"150,000km",6,benzin,bmw,nein,2016-04-04 00:00:00,0,71034,2016-04-06 14:45:08
2,2016-03-26 18:57:24,Volkswagen_Golf_1.6_United,privat,Angebot,"$8,990",test,limousine,2009,manuell,102,golf,"70,000km",7,benzin,volkswagen,nein,2016-03-26 00:00:00,0,35394,2016-04-06 20:15:37
3,2016-03-12 16:58:10,Smart_smart_fortwo_coupe_softouch/F1/Klima/Pan...,privat,Angebot,"$4,350",control,kleinwagen,2007,automatik,71,fortwo,"70,000km",6,benzin,smart,nein,2016-03-12 00:00:00,0,33729,2016-03-15 03:16:28
4,2016-04-01 14:38:50,Ford_Focus_1_6_Benzin_TÜV_neu_ist_sehr_gepfleg...,privat,Angebot,"$1,350",test,kombi,2003,manuell,0,focus,"150,000km",7,benzin,ford,nein,2016-04-01 00:00:00,0,39218,2016-04-01 14:38:50


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

Upon our initial inspection, we see that there is roughly 50,000 records across 20 columns featuring some null values and a mix of strings and numerics.

## Data Cleaning
### Part 1: Column Cleaning

The first task in our data cleaning will be to rename the columns. Below, we see that most of the column names are camelCased while some are overly long.

In [4]:
df.columns

Index(['dateCrawled', 'name', 'seller', 'offerType', 'price', 'abtest',
       'vehicleType', 'yearOfRegistration', 'gearbox', 'powerPS', 'model',
       'odometer', 'monthOfRegistration', 'fuelType', 'brand',
       'notRepairedDamage', 'dateCreated', 'nrOfPictures', 'postalCode',
       'lastSeen'],
      dtype='object')

Here, we'll use the **Dataframe.rename()** method to pass a dictionary that simplifies some column names while changing camelCased names to snake_cased. The **Axis=1** argument notes the column will be affected, while **inplace=True** will swap one string for the other.

In [5]:
df.rename({
    "dateCrawled": "date_crawled",
    "offerType": "offer_type",
    "vehicleType": "vehicle_type",
    "yearOfRegistration": "registration_year",
    "powerPS": "power_ps",
    "monthOfRegistration": "registration_month",
    "fuelType": "fuel_type",
    "notRepairedDamage": "unrepaired_damage",
    "dateCreated": "ad_created",
    "nrOfPictures": "pictures",
    "postalCode": "postal_code",
    "lastSeen": "last_seen"}, axis=1, inplace=True)

With the use of **Dataframe.describe()**, we learn even more about our dataset.

In [6]:
df.describe(include='all')

,date_crawled,name,seller,offer_type,price,abtest,vehicle_type,registration_year,gearbox,power_ps,model,odometer,registration_month,fuel_type,brand,unrepaired_damage,ad_created,pictures,postal_code,last_seen
count,50000,50000,50000,50000,50000,50000,44905,50000.000000,47320,50000.000000,47242,50000,50000.000000,45518,50000,40171,50000,50000.0,50000.000000,50000
unique,48213,38754,2,2,2357,2,8,NaN,2,NaN,245,13,NaN,7,40,2,76,NaN,NaN,39481
top,2016-03-22 09:51:06,Ford_Fiesta,privat,Angebot,$0,test,limousine,NaN,manuell,NaN,golf,"150,000km",NaN,benzin,volkswagen,nein,2016-04-03 00:00:00,NaN,NaN,2016-04-07 06:17:27
freq,3,78,49999,49999,1421,25756,12859,NaN,36993,NaN,4024,32424,NaN,30107,10687,35232,1946,NaN,NaN,8
mean,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2005.073280,NaN,116.355920,NaN,NaN,5.723360,NaN,NaN,NaN,NaN,0.0,50813.627300,NaN
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN,105.712813,NaN,209.216627,NaN,NaN,3.711984,NaN,NaN,NaN,NaN,0.0,25779.747957,NaN
min,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1000.000000,NaN,0.000000,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,0.0,1067.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1999.000000,NaN,70.000000,NaN,NaN,3.000000,NaN,NaN,NaN,NaN,0.0,30451.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2003.000000,NaN,105.000000,NaN,NaN,6.000000,NaN,NaN,NaN,NaN,0.0,49577.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2008.000000,NaN,150.000000,NaN,NaN,9.000000,NaN,NaN,NaN,NaN,0.0,71540.000000,NaN


Meanwhile, other object-based columns like "odometer" and "price" should be recategorized as numeric after reformatting such values to omit symbols and non numeric characters. We'll use **Series.str.replace** to remove $, km, and commas from each column's values, then convert them to numeric types with **Series.astype()**.

In [7]:
df.rename({"odometer":"odometer_km"}, axis=1, inplace=True)

In [8]:
df["odometer_km"] = df["odometer_km"].str.replace("km", "").str.replace(",","").astype(int)
df["price"] = df["price"].str.replace("$","").str.replace(",","").astype(int)

### Part 2: Removing Outliers

Next, we'll search for outliers in our data. Targeting "price" and "odometer_km" for such outliers is a good place to start as numberics with great potential range to skew the data.  We'll use **Series.sort_values()** to retrieve extremes on either end of each column.

Upon inspection of "odometer_km", it seems that the bulk of cars that go up for sale are done after heavy use. With only 13 unique values with somewhat even distribution, we'll leave this column intact.

In [9]:
df["odometer_km"].value_counts()

150000    32424
125000     5170
100000     2169
90000      1757
80000      1436
70000      1230
60000      1164
50000      1027
5000        967
40000       819
30000       789
20000       784
10000       264
Name: odometer_km, dtype: int64

Meanwhile, prices in this dataset range from 0 to \$1B. We'll slice the top 14 records after \$350,000 as it's the biggest jump of $600,000 we'll see in this data set as well as 0 values as cars are never given away freely.

In [10]:
df["price"].value_counts().sort_index(ascending=False).head(20)

99999999    1
27322222    1
12345678    3
11111111    2
10000000    1
3890000     1
1300000     1
1234566     1
999999      2
999990      1
350000      1
345000      1
299000      1
295000      1
265000      1
259000      1
250000      1
220000      1
198000      1
197000      1
Name: price, dtype: int64

In [11]:
df = df[df["price"].between(1,350000)]

To understand the nature of the data gathered, we'll investigate the distribution of values as a percentage with **Series.value_counts(normalize=True)** after slicing the date string from each columns with **Series.str**.

What we see here is that the dataset represents roughly uniformed crawled data from a month period between March and April 2016.

In [13]:
df["date_crawled"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2016-03-05    0.025327
2016-03-06    0.014043
2016-03-07    0.036014
2016-03-08    0.033296
2016-03-09    0.033090
2016-03-10    0.032184
2016-03-11    0.032575
2016-03-12    0.036920
2016-03-13    0.015670
2016-03-14    0.036549
2016-03-15    0.034284
2016-03-16    0.029610
2016-03-17    0.031628
2016-03-18    0.012911
2016-03-19    0.034778
2016-03-20    0.037887
2016-03-21    0.037373
2016-03-22    0.032987
2016-03-23    0.032225
2016-03-24    0.029342
2016-03-25    0.031607
2016-03-26    0.032204
2016-03-27    0.031092
2016-03-28    0.034860
2016-03-29    0.034099
2016-03-30    0.033687
2016-03-31    0.031834
2016-04-01    0.033687
2016-04-02    0.035478
2016-04-03    0.038608
2016-04-04    0.036487
2016-04-05    0.013096
2016-04-06    0.003171
2016-04-07    0.001400
Name: date_crawled, dtype: float64

From the "ad_created" column, we see the bulk of the ads were created within the timeframe of the data crawl. Some ads date as far back as 9 months prior to the crawl period.

In [20]:
df["ad_created"].str[:10].value_counts(normalize=True, dropna=False).sort_index()

2015-06-11    0.000021
2015-08-10    0.000021
2015-09-09    0.000021
2015-11-10    0.000021
2015-12-05    0.000021
2015-12-30    0.000021
2016-01-03    0.000021
2016-01-07    0.000021
2016-01-10    0.000041
2016-01-13    0.000021
2016-01-14    0.000021
2016-01-16    0.000021
2016-01-22    0.000021
2016-01-27    0.000062
2016-01-29    0.000021
2016-02-01    0.000021
2016-02-02    0.000041
2016-02-05    0.000041
2016-02-07    0.000021
2016-02-08    0.000021
2016-02-09    0.000021
2016-02-11    0.000021
2016-02-12    0.000041
2016-02-14    0.000041
2016-02-16    0.000021
2016-02-17    0.000021
2016-02-18    0.000041
2016-02-19    0.000062
2016-02-20    0.000041
2016-02-21    0.000062
                ...   
2016-03-09    0.033151
2016-03-10    0.031895
2016-03-11    0.032904
2016-03-12    0.036755
2016-03-13    0.017008
2016-03-14    0.035190
2016-03-15    0.034016
2016-03-16    0.030125
2016-03-17    0.031278
2016-03-18    0.013590
2016-03-19    0.033687
2016-03-20    0.037949
2016-03-21 

Inspection of the "registration_year" columns shows values of 1000 (before cars were invented) and 9999 (after the 2016 cutoff) within the dataset that will need to be addressed.

In [22]:
df["registration_year"].describe()

count    48565.000000
mean      2004.755421
std         88.643887
min       1000.000000
25%       1999.000000
50%       2004.000000
75%       2008.000000
max       9999.000000
Name: registration_year, dtype: float64

With the values outside 1900 and 2016 accounting for less than 4% of the data, we'll slice them out of the final dataframe and begin our analysis of the most popular cars on ebay Germany.

In [45]:
(~df["registration_year"].between(1900,2016)).sum() / df.shape[0]

0.038793369710697

In [50]:
df = df[df["registration_year"].between(1900,2016)]

## Analysis

### Part 1: Most Common Year

Many of the cars for sale were first registered in the past 20 years.

In [49]:
df["registration_year"].value_counts(normalize=True).head(10)

2000    0.067608
2005    0.062895
1999    0.062060
2004    0.057904
2003    0.057818
2006    0.057197
2001    0.056468
2002    0.053255
1998    0.050620
2007    0.048778
Name: registration_year, dtype: float64

### Part 2: Most Common Brand

Volkswagen is the most commonly found brand of car for sale, followed by BMW and Opel.

In [69]:
df["brand"].value_counts(normalize=True).head(10)

volkswagen       0.211264
bmw              0.110045
opel             0.107581
mercedes_benz    0.096463
audi             0.086566
ford             0.069900
renault          0.047150
peugeot          0.029841
fiat             0.025642
seat             0.018273
Name: brand, dtype: float64

### Part 3: Average Price for Top Brand

The following code will calculate the average prices for all brands in the dataset.

In [71]:
avg_brand_price = {}

all_brands = df["brand"].unique()
for brand in all_brands:
    selection = df[df["brand"] == brand]
    average_price = (selection["price"].sum() / selection.shape[0])
    avg_brand_price[brand] = float("{:.2f}".format(average_price))

Meanwhile, this will filter the master list down to the top 5 brands.

In [72]:
top_5_brands = ["volkswagen", "bmw", "opel", "mercedes_benz", "audi"]
top_5_prices = {}

for brand in top_5_brands:
    top_5_prices[brand] = avg_brand_price[brand]

top_5_prices

{'audi': 9336.69,
 'bmw': 8332.82,
 'mercedes_benz': 8628.45,
 'opel': 2975.24,
 'volkswagen': 5402.41}

### Part 4: Average Miles Per Top Brand

In [80]:
average_brand_odometer = {}

for brand in top_5_brands:
    selection = df[df["brand"] == brand]
    average_odometer = (selection["odometer_km"].sum() / selection.shape[0])
    average_brand_odometer[brand] = average_odometer.round()

average_brand_odometer

{'audi': 129157.0,
 'bmw': 132573.0,
 'mercedes_benz': 130788.0,
 'opel': 129310.0,
 'volkswagen': 128707.0}

In [81]:
abo = pd.Series(average_brand_odometer)
abp = pd.Series(top_5_prices)

In [91]:
finalframe = pd.DataFrame(abo, columns=["mean_mileage"])
finalframe["mean_price"] = abp
finalframe

,mean_mileage,mean_price
audi,129157.0,9336.69
bmw,132573.0,8332.82
mercedes_benz,130788.0,8628.45
opel,129310.0,2975.24
volkswagen,128707.0,5402.41


Despite differences in price of luxury brands and consumer brands, cars care commonly sold after 100,000km of use.